# Load the dataset

In [5]:
!kaggle datasets download -d sixhky/open-images-bus-trucks -p ./Dataset
!unzip ./Dataset/open-images-bus-trucks.zip -d ./Dataset

Dataset URL: https://www.kaggle.com/datasets/sixhky/open-images-bus-trucks
License(s): unknown
open-images-bus-trucks.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./Dataset/open-images-bus-trucks.zip
  inflating: ./Dataset/df.csv        
  inflating: ./Dataset/images/images/0000599864fd15b3.jpg  
  inflating: ./Dataset/images/images/00006bdb1eb5cd74.jpg  
  inflating: ./Dataset/images/images/00010bf498b64bab.jpg  
  inflating: ./Dataset/images/images/00013f14dd4e168f.jpg  
  inflating: ./Dataset/images/images/0002914fa805e227.jpg  
  inflating: ./Dataset/images/images/00036603fa39fa85.jpg  
  inflating: ./Dataset/images/images/0004d5a9dd44ab6a.jpg  
  inflating: ./Dataset/images/images/0005f203463a13a8.jpg  
  inflating: ./Dataset/images/images/00066517f9d814f9.jpg  
  inflating: ./Dataset/images/images/00072b81abc72d21.jpg  
  inflating: ./Dataset/images/images/000780f791b71252.jpg  
  inflating: ./Dataset/images/images/0007eeeabf3c5

In [6]:
from torchvision import transforms, models, datasets
from torch_snippets import Report
from torchvision.ops import nms # Non-Maximum Suppression


ModuleNotFoundError: No module named 'torchvision'